<a href="https://colab.research.google.com/github/jooalee64/mastertip/blob/main/mastertip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [41]:
boats_dir = '/gdrive/MyDrive/boats'
%cd $boats_dir
print(f"Current directory: {boats_dir}")

/gdrive/.shortcut-targets-by-id/1-rAOCAOs7fjfKOgyVrhuYl0x8viIZLuX/boats
Current directory: /gdrive/MyDrive/boats


In [53]:
from PIL import Image
import os

# Assuming boats_dir is defined from the previous cell (3yxUNutZD6EY)
# Use the specific directory path provided by the user
images_to_load_dir = '/gdrive/MyDrive/boats/John/Boat Photos'


loaded_images = []
image_paths = [] # Also update image_paths to reflect the loaded images

try:
    # List all files in the directory
    all_files = os.listdir(images_to_load_dir)

    # Filter for image files (you might want to add more image extensions)
    image_files = [f for f in all_files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Construct the full paths and load the images
    for f in image_files:
        img_path = os.path.join(images_to_load_dir, f)
        image_paths.append(img_path) # Add the path to image_paths
        try:
            img = Image.open(img_path).convert("RGB") # Ensure image is in RGB format
            loaded_images.append(img)
            # print(f"Successfully loaded: {img_path}")
        except FileNotFoundError:
            print(f"Error: Image file not found at {img_path}")
        except Exception as e:
            print(f"An error occurred while loading {img_path}: {e}")

except FileNotFoundError:
    print(f"Error: Directory not found at {images_to_load_dir}")
except Exception as e:
    print(f"An error occurred while listing files in {images_to_load_dir}: {e}")


if loaded_images:
    print(f"Successfully loaded {len(loaded_images)} images.")
else:
    print("No images were loaded.")

Successfully loaded 155 images.


In [54]:
from PIL import Image
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torchvision.models as models

# Define the MastTipDetectionModel class (included here to ensure it's defined before use)
class MastTipDetectionModel(nn.Module):
    def __init__(self, num_classes=1):
        super(MastTipDetectionModel, self).__init__()
        # Using a pre-trained ResNet as a backbone (example)
        self.backbone = models.resnet50(pretrained=True)
        # Modify the classifier for bounding box regression, mast tip prediction, and uncertainty
        num_ftrs = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity() # Remove the original classification layer

        # Add custom layers for the outputs
        # Bounding box: 4 coordinates (x1, y1, x2, y2)
        self.bbox_predictor = nn.Linear(num_ftrs, 4)
        # Mast tip: 2 coordinates (x, y)
        self.mast_tip_predictor = nn.Linear(num_ftrs, 2)
        # Uncertainty: 1 value (scalar)
        self.uncertainty_predictor = nn.Linear(num_ftrs, 1)

    def forward(self, x):
        # Forward pass through the backbone
        features = self.backbone(x)

        # Predict outputs
        bbox = self.bbox_predictor(features)
        mast_tip = self.mast_tip_predictor(features)
        uncertainty = self.uncertainty_predictor(features)

        return bbox, mast_tip, uncertainty


# Assuming image_paths is already defined from the previous cell (auNVzZytQUHo)
# Define preprocessing transformations
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to a fixed size (e.g., 224x224 for ResNet)
    transforms.ToTensor(),         # Convert PIL Image to PyTorch Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # ImageNet normalization
])

# Load and preprocess the images
processed_images = []
for img_path in image_paths:
    try:
        img = Image.open(img_path).convert("RGB") # Ensure image is in RGB format
        img_tensor = preprocess(img)
        processed_images.append(img_tensor)
        print(f"Successfully processed: {img_path}")
    except FileNotFoundError:
        print(f"Error: Image file not found at {img_path}")
    except Exception as e:
        print(f"An error occurred while processing {img_path}: {e}")

# Stack the processed images into a batch tensor
if processed_images:
    batch_tensor = torch.stack(processed_images)
    print(f"Created a batch tensor of shape: {batch_tensor.shape}")

    # Instantiate the model
    model = MastTipDetectionModel()
    # Load pre-trained weights if available
    # model.load_state_dict(torch.load('your_model_weights.pth'))
    model.eval() # Set the model to evaluation mode

    # Perform inference
    with torch.no_grad(): # Disable gradient calculation
        bbox_predictions, mast_tip_predictions, uncertainty_predictions = model(batch_tensor)

    # Print or use the predictions as needed
    print("Bounding box predictions:", bbox_predictions)
    print("Mast tip predictions:", mast_tip_predictions)
    print("Uncertainty predictions:", uncertainty_predictions)

else:
    print("No images were successfully processed to run inference.")

Successfully processed: /gdrive/MyDrive/boats/John/Boat Photos/1572.jpg
Successfully processed: /gdrive/MyDrive/boats/John/Boat Photos/1573.jpg
Successfully processed: /gdrive/MyDrive/boats/John/Boat Photos/1596.jpg
Successfully processed: /gdrive/MyDrive/boats/John/Boat Photos/1567.jpg
Successfully processed: /gdrive/MyDrive/boats/John/Boat Photos/1554.jpg
Successfully processed: /gdrive/MyDrive/boats/John/Boat Photos/1627.jpg
Successfully processed: /gdrive/MyDrive/boats/John/Boat Photos/1625.jpg
Successfully processed: /gdrive/MyDrive/boats/John/Boat Photos/1592.jpg
Successfully processed: /gdrive/MyDrive/boats/John/Boat Photos/1636.jpg
Successfully processed: /gdrive/MyDrive/boats/John/Boat Photos/1630.jpg
Successfully processed: /gdrive/MyDrive/boats/John/Boat Photos/1560.jpg
Successfully processed: /gdrive/MyDrive/boats/John/Boat Photos/1612.jpg
Successfully processed: /gdrive/MyDrive/boats/John/Boat Photos/1615.jpg
Successfully processed: /gdrive/MyDrive/boats/John/Boat Photos/1

In [55]:
# from PIL import Image
# import torchvision.transforms as transforms
# import torch

# # Assuming image_paths is already defined from the previous cell (auNVzZytQUHo)
# # Define preprocessing transformations
# preprocess = transforms.Compose([
#     transforms.Resize((224, 224)),  # Resize to a fixed size (e.g., 224x224 for ResNet)
#     transforms.ToTensor(),         # Convert PIL Image to PyTorch Tensor
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # ImageNet normalization
# ])

# # Load and preprocess the images
# processed_images = []
# for img_path in image_paths:
#     try:
#         img = Image.open(img_path).convert("RGB") # Ensure image is in RGB format
#         img_tensor = preprocess(img)
#         processed_images.append(img_tensor)
#         print(f"Successfully processed: {img_path}")
#     except FileNotFoundError:
#         print(f"Error: Image file not found at {img_path}")
#     except Exception as e:
#         print(f"An error occurred while processing {img_path}: {e}")

# # Stack the processed images into a batch tensor
# if processed_images:
#     batch_tensor = torch.stack(processed_images)
#     print(f"Created a batch tensor of shape: {batch_tensor.shape}")

#     # Instantiate the model (assuming MastTipDetectionModel is defined in a previous cell)
#     model = MastTipDetectionModel()
#     # Load pre-trained weights if available
#     # model.load_state_dict(torch.load('your_model_weights.pth'))
#     model.eval() # Set the model to evaluation mode

#     # Perform inference
#     with torch.no_grad(): # Disable gradient calculation
#         bbox_predictions, mast_tip_predictions, uncertainty_predictions = model(batch_tensor)

#     # Print or use the predictions as needed
#     print("Bounding box predictions:", bbox_predictions)
#     print("Mast tip predictions:", mast_tip_predictions)
#     print("Uncertainty predictions:", uncertainty_predictions)

# else:
#     print("No images were successfully processed to run inference.")